In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
df  = pd.read_csv('./data.csv')
df.head()

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,...,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,"17,50,000",acquired,USA,NY,New York City,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,"40,00,000",operating,USA,CA,Los Angeles,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Publishing,"40,000",operating,EST,NaN,Tallinn,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,/organization/in-touch-network,(In)Touch Network,http://www.InTouchNetwork.com,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,"15,00,000",operating,GBR,NaN,London,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,"60,000",operating,USA,TX,Dallas,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49438 entries, 0 to 49437
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   permalink             49438 non-null  object 
 1   name                  49437 non-null  object 
 2   homepage_url          45987 non-null  object 
 3   category_list         45475 non-null  object 
 4    market               45468 non-null  object 
 5    funding_total_usd    49434 non-null  object 
 6   status                48120 non-null  object 
 7   country_code          44162 non-null  object 
 8   state_code            30159 non-null  object 
 9   region                44162 non-null  object 
 10  city                  43319 non-null  object 
 11  funding_rounds        49434 non-null  float64
 12  founded_at            38551 non-null  object 
 13  founded_month         38479 non-null  object 
 14  founded_quarter       38479 non-null  object 
 15  founded_year       

In [10]:
df.rename(columns={' funding_total_usd ': "funding_total_usd",
                    ' market ': "market"},inplace=True)

### Missing data

In [11]:
CATEGORICAL = list(filter(lambda x: df[x].dtype=='object',df.columns))
def show_stats(columns):
    stat = {}
    for col in columns:
        stat[col] =  [df[col].nunique(), df[col].isna().sum(), df[col].dtype]
    return pd.DataFrame.from_dict(stat, orient='index', columns=['Unique',  'Missed values in %', 'DType'])
show_stats(CATEGORICAL)

,Unique,Missed values in %,DType
permalink,49436,0,object
name,49349,1,object
homepage_url,45848,3451,object
category_list,16674,3963,object
market,753,3970,object
funding_total_usd,14617,4,object
status,3,1318,object
country_code,115,5276,object
state_code,61,19279,object
region,1089,5276,object


cột 'funding_total_usd' đáng nhẽ là dạng số nhưng có dấu phẩy nên phải covert về numeric

In [12]:
df['funding_total_usd'].head()

0     17,50,000 
1     40,00,000 
2         40,000
3     15,00,000 
4         60,000
Name: funding_total_usd, dtype: object

In [13]:
def str_to_int(s: str):
    try:
        s = s.replace(',', '')
        if '-' in s:
            s = 0
        return int(s)
    except ValueError:
        return s
    except AttributeError:
        return s
df['funding_total_usd'] = df['funding_total_usd'].apply(str_to_int)

In [14]:
df = df[df['funding_total_usd']!=0]

covert các cột datetime

In [15]:
df[['founded_at','first_funding_at','last_funding_at']].head()

,founded_at,first_funding_at,last_funding_at
0,01/06/2012,30/06/2012,30/06/2012
1,NaN,04/06/2010,23/09/2010
2,26/10/2012,09/08/2012,09/08/2012
3,01/04/2011,01/04/2011,01/04/2011
4,01/01/2014,17/08/2014,26/09/2014


In [16]:
format_date = ('founded_at','first_funding_at','last_funding_at')
for i in format_date:
  df[i] = pd.to_datetime(df[i], errors = 'coerce')

C:\Users\PC\AppData\Local\Temp\ipykernel_2592\425228424.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[i] = pd.to_datetime(df[i], errors = 'coerce')
C:\Users\PC\AppData\Local\Temp\ipykernel_2592\425228424.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[i] = pd.to_datetime(df[i], errors = 'coerce')


In [17]:
#Để giảm độ phức tạp, trong khi xử lý các trường biến ngày, chuyển đổi ngày thực tế thành ngày tương đối, mô tả khoảng cách từ khi thành lập đến lần cấp vốn đầu tiên và cuối cùng, sau đó bỏ ngày thành lập.

df['age_first_funding'] = (df['first_funding_at']-df['founded_at'])/pd.Timedelta(days=365)
df['age_last_funding'] = (df['last_funding_at']-df['founded_at'])/pd.Timedelta(days=365)
df['founded_month'] = pd.DatetimeIndex(df['founded_at']).month
df['founded_quarter'] = pd.DatetimeIndex(df['founded_at']).quarter

In [18]:
df.isnull().sum()

permalink                   0
name                        1
homepage_url             2316
category_list            2506
market                   2511
funding_total_usd           4
status                   1109
country_code             3822
state_code              15290
region                   3822
city                     4508
funding_rounds              4
founded_at              12007
founded_month           12007
founded_quarter         12007
founded_year             8775
first_funding_at            6
last_funding_at             4
seed                        4
venture                     4
equity_crowdfunding         4
undisclosed                 4
convertible_note            4
debt_financing              4
angel                       4
grant                       4
private_equity              4
post_ipo_equity             4
post_ipo_debt               4
secondary_market            4
product_crowdfunding        4
round_A                     4
round_B                     4
round_C   

xoá các cột không cần thiết

In [19]:
df = df.drop(['permalink','homepage_url' ,'state_code','region','city','founded_at','first_funding_at','funding_total_usd','last_funding_at','category_list' ], axis = 1)

xoá tất cả các dupplicate 

In [20]:
df = df.drop_duplicates()
df.shape

(40908, 31)

Xu li cac cut numeric

In [21]:
show_stats(list(filter(lambda x: df[x].dtype=='float64',df.columns)))

,Unique,Missed values in %,DType
funding_rounds,17,3,float64
founded_month,12,12006,float64
founded_quarter,4,12006,float64
founded_year,101,8774,float64
seed,3337,3,float64
venture,9300,3,float64
equity_crowdfunding,252,3,float64
undisclosed,687,3,float64
convertible_note,299,3,float64
debt_financing,1872,3,float64


In [22]:
df[['founded_month','founded_quarter','founded_year']].head()

,founded_month,founded_quarter,founded_year
0,1.0,1.0,2012.0
1,NaN,NaN,NaN
2,NaN,NaN,2012.0
3,1.0,1.0,2011.0
4,1.0,1.0,2014.0


41k data là khá lớn, hoàn toàn có thể drop null

In [23]:
df = df.dropna()
df.shape

(25499, 31)

In [24]:
df.isnull().sum().sum()

0

Data Transformation

In [25]:
df.head()

,name,market,status,country_code,funding_rounds,founded_month,founded_quarter,founded_year,seed,venture,...,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,age_first_funding,age_last_funding
0,#waywire,News,acquired,USA,1.0,1.0,1.0,2012.0,1750000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.482192,0.482192
3,(In)Touch Network,Electronics,operating,GBR,1.0,1.0,1.0,2011.0,1500000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.238356,0.238356
4,-R- Ranch and Mine,Tourism,operating,USA,2.0,1.0,1.0,2014.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.624658,0.734247
10,"1,2,3 Listo",E-Commerce,operating,CHL,1.0,1.0,1.0,2012.0,40000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.134247,1.134247
13,1-800-DOCTORS,Health and Wellness,operating,USA,1.0,1.0,1.0,1984.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.183562,27.183562


Xem giá trị khác nhau của từng cột

In [27]:

num_unique_country_code = df['country_code'].nunique()
num_unique_market = df['market'].nunique()
num_unique_status = df['status'].nunique()

print("Số lượng giá trị khác nhau trong cột 'market':", num_unique_market)
print("Số lượng giá trị khác nhau trong cột 'country_code':", num_unique_country_code)
print("Số lượng giá trị khác nhau trong cột 'status':", num_unique_status)

Số lượng giá trị khác nhau trong cột 'market': 673
Số lượng giá trị khác nhau trong cột 'country_code': 104
Số lượng giá trị khác nhau trong cột 'status': 3


Hiển thị phân phối của các giá trị

In [28]:

market_distribution = df['market'].value_counts().reset_index()
market_distribution.columns = ['market', 'count']
print("\nPhân phối của các giá trị trong cột 'market':\n", market_distribution)


Phân phối của các giá trị trong cột 'market':
                                 market  count
0                            Software    2881
1                       Biotechnology    2243
2                              Mobile    1118
3                          E-Commerce     916
4                         Curated Web     857
..                                 ...    ...
668         Human Resource Automation       1
669                         Teenagers       1
670   Biotechnology and Semiconductor       1
671              Knowledge Management       1
672                              Wind       1

[673 rows x 2 columns]


Gom nhóm các giá trị ít xuất hiện

In [29]:
# Chọn ngưỡng
threshold = 200

# Lấy các giá trị hiếm xuất hiện
rare_values = df['market'].value_counts()[df['market'].value_counts() < threshold].index

# Gom nhóm các giá trị hiếm xuất hiện lại thành 'Others'
df.loc[df['market'].isin(rare_values), 'market'] = 'Others'


In [30]:
df.head()

,name,market,status,country_code,funding_rounds,founded_month,founded_quarter,founded_year,seed,venture,...,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,age_first_funding,age_last_funding
0,#waywire,Others,acquired,USA,1.0,1.0,1.0,2012.0,1750000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.482192,0.482192
3,(In)Touch Network,Others,operating,GBR,1.0,1.0,1.0,2011.0,1500000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.238356,0.238356
4,-R- Ranch and Mine,Others,operating,USA,2.0,1.0,1.0,2014.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.624658,0.734247
10,"1,2,3 Listo",E-Commerce,operating,CHL,1.0,1.0,1.0,2012.0,40000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.134247,1.134247
13,1-800-DOCTORS,Health and Wellness,operating,USA,1.0,1.0,1.0,1984.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.183562,27.183562


Xét cột country_code

In [31]:
country_code_distribution = df['country_code'].value_counts().reset_index()
country_code_distribution.columns = ['country_code', 'count']
print("\nPhân phối của các giá trị trong cột 'country_code':\n", country_code_distribution)


Phân phối của các giá trị trong cột 'country_code':
     country_code  count
0            USA  17593
1            GBR   1479
2            CAN    754
3            FRA    499
4            CHN    482
..           ...    ...
99           MCO      1
100          AZE      1
101          GTM      1
102          NIC      1
103          MAF      1

[104 rows x 2 columns]


Chọn ngưỡng

In [32]:
# Chọn ngưỡng
threshold = 100

# Lấy các giá trị hiếm xuất hiện
rare_values = df['country_code'].value_counts()[df['country_code'].value_counts() < threshold].index

# Gom nhóm các giá trị hiếm xuất hiện lại thành 'Others'
df.loc[df['country_code'].isin(rare_values), 'country_code'] = 'Remaining'